<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/neural_network_OOP_241021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perceptron -- neuron with an activation function.
Inputs and their weights are numbered 0 to(n-1), bias is n

In [4]:
import numpy as np

In [2]:
class Perceptron:
    """A single neuron with the sigmoid activation function.
       Attributes:
          inputs: The number of inputs in the perceptron, not counting the bias.
          bias:   The bias term. By defaul it's 1.0."""

    def __init__(self, inputs, bias = 1.0):
        """Return a new Perceptron object with the specified number of inputs (+1 for the bias).""" 
        self.weights = (np.random.rand(inputs+1) * 2) - 1 
        self.bias = bias

    def run(self, x):
        """Run the perceptron. x is a python list with the input values."""
        sum = np.dot(np.append(x,self.bias),self.weights)
        return self.sigmoid(sum)

    def set_weights(self, w_init):
        # w_init is a list of floats. Organize it as you'd like.
        self.w_init=np.array(w_init)

    def sigmoid(self, x):
        # return the output of the sigmoid function applied to x
        return 1/(1+np.exp(-x))